In [ ]:
from lm.create_lm import *
# path to KenLM binary
lm_path = "/home/daniel/KerasDeepSpeech/lm/libri-timit-lm.klm"
# path to vocabulary
vocab_path = "/home/daniel/KerasDeepSpeech/lm/words.txt"

check_lm(lm_path, vocab_path, "language modelling is fun")

In [ ]:
from lm.create_lm import *
import kenlm    
# path to KenLM binary
lm_path = "../lm/wiki_de/wiki_de_2_gram.klm"
# path to vocabulary
vocab_path = "../lm/wiki_de/wiki_de.words"

model = kenlm.LanguageModel(lm_path)
print(f'loaded {model.order}-gram model from {lm_path}')

for sentence in ["Ich bin ein Berliner", "Berliner ein ich bin"]:
    print(sentence)
    print("score:", model.score(sentence))
    for prob, ngram_length, oov in model.full_scores(sentence):
        print({'probability': prob, "n-gram lenght": ngram_length, "oov?": oov})
    print("perplexity:", model.perplexity(sentence))
    print()

In [ ]:
lm_path = '../lm/wiki_de/srilm-voxforge-de-r20171217.klm'
import kenlm
model = kenlm.LanguageModel(lm_path)
print(f'loaded {model.order}-gram model from {lm_path}')
print(model.score('Wir vergleichen den Wert eines korrekten Satzes mit dem Wert eines fehlerhaften Satzes.'))
print(model.score('Wir verglaichen den Wirt eines korrekten Sitzes mit dem Wort eines fehlerhaften Sätzes.'))